In [38]:
import duckdb

# Ensure data has been added to expected directory as per README instructions
YTM_FILEPATH = "/workspaces/ytmusic_analytics/data/watch-history.json"

In [39]:
try:
    data = duckdb.read_json(YTM_FILEPATH)
except FileNotFoundError as e:
    print(f"File not found: {YTM_FILEPATH}. Please ensure the file exists.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [40]:
data.describe()

┌─────────┬───────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────┬───────────┬─────────────────────────────────────────────┬─────────────────────┬─────────────────────────────┐
│  aggr   │    header     │                                                                                                                         title                                                                                                                          │                   titleUrl                    │                                                         subtitles                       

In [41]:
# Create table
q = """
CREATE OR REPLACE TABLE watch_history_clean AS
SELECT
  header,
  CASE
    WHEN title LIKE 'Watched %'
      THEN substr(title, length('Watched ')+1)
    ELSE title
  END AS title,
  titleUrl,
  subtitles,
  CAST("time" AS TIMESTAMP) AS ts,
  products,
  activityControls,
  description,
  details
FROM data;
"""

duckdb.sql(q)

In [42]:
# Validate
q = """SELECT * 
FROM watch_history_clean
LIMIT 100"""
r = duckdb.sql(q)
r.show()

┌───────────────┬──────────────────────────────────────────┬───────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────┬───────────┬─────────────────────────┬─────────────┬──────────────────────────┐
│    header     │                  title                   │                   titleUrl                    │                                                   subtitles                                                   │           ts            │ products  │    activityControls     │ description │         details          │
│    varchar    │                 varchar                  │                    varchar                    │                                     struct("name" varchar, url varchar)[]                                     │        timestamp        │ varchar[] │        varchar[]        │   varchar   │ struct("name" varchar)[] │
├───────────────┼─────

In [43]:
# Query to count rows before removal
row_count_before_query = """
SELECT COUNT(*)
FROM watch_history_clean
"""

row_count_before = duckdb.sql(row_count_before_query).fetchone()[0]
print(f"Row count before removal: {row_count_before}")

# Query to identify rows to be removed
count_rows_with_ads_query = """
SELECT COUNT(*) AS before_count
FROM watch_history_clean
WHERE len(details) = 1
  AND details[1].name = 'From Google Ads';
"""

ad_rows_to_be_removed = duckdb.sql(count_rows_with_ads_query).fetchone()[0]
print(f"Rows identified for removal: {ad_rows_to_be_removed}")

# Query to delete the matching ads from the table
remove_ads_query = """
DELETE FROM watch_history_clean
WHERE len(details) = 1
  AND details[1].name = 'From Google Ads';
"""

# Execute the deletion
duckdb.sql(remove_ads_query)

# Query to count rows after removal
count_after_removal_query = """
SELECT COUNT(*) AS after_count
FROM watch_history_clean
"""

row_count_after = duckdb.sql(count_after_removal_query).fetchone()[0]

# Calculate the number of rows removed
rows_removed_actual = row_count_before - row_count_after

# Output the results
print(f"Rows removed expected: {ad_rows_to_be_removed}")
print(f"Rows removed actual: {rows_removed_actual}")
print(f"Rows after removal: {row_count_after}")


Row count before removal: 17634
Rows identified for removal: 87
Rows removed expected: 87
Rows removed actual: 87
Rows after removal: 17547


In [44]:
# Create 2025 table
q = """
CREATE OR REPLACE TABLE yt_music_history_2025 AS
SELECT
  ROW_NUMBER() OVER (ORDER BY ts)      AS id,
  title                                  AS song_title,
  --Trim topic
  IF(
    -- check if the last 8 chars are ' - Topic'
    subtitles[1].name[-8:] = ' - Topic',
    -- if so, drop those 8 chars
    subtitles[1].name[:-8],
    -- otherwise leave untouched
    subtitles[1].name) 					 AS song_artist,
  CAST(ts AS TIMESTAMP)                  AS listened_ts,
  titleUrl                               AS youtube_url
FROM watch_history_clean
WHERE ts >= '2025-01-01'
and "header" = 'YouTube Music'"""

duckdb.sql(q)


In [45]:
# Validate
q = """SELECT *
FROM yt_music_history_2025
LIMIT 100"""

r = duckdb.sql(q)
r.show()

┌───────┬───────────────────────────────────────────────────┬──────────────────┬─────────────────────────┬───────────────────────────────────────────────┐
│  id   │                    song_title                     │   song_artist    │       listened_ts       │                  youtube_url                  │
│ int64 │                      varchar                      │     varchar      │        timestamp        │                    varchar                    │
├───────┼───────────────────────────────────────────────────┼──────────────────┼─────────────────────────┼───────────────────────────────────────────────┤
│     1 │ Hades II - The Crossroads                         │ Supergiant Games │ 2025-01-01 22:48:14.432 │ https://music.youtube.com/watch?v=f5-XklYU0Xw │
│     2 │ Hades II - Lost Souls                             │ Supergiant Games │ 2025-01-01 22:52:39.603 │ https://music.youtube.com/watch?v=VhPausM5MOs │
│     3 │ Hades II - Spider Silk                            │ Supergia